In [1]:
import pandas as pd
import datetime

We want to get a network file from LANL, open it in Pandas (only the first 500,000 rows - which is only about 2.5 minutes of data), convert time to actual timestamps, and then output in streaming format JSON, in sets of 10,000 rows. So only 50 JSON files. You can adjust if you'd like but 50 should be enough to move through quickly.

In [ ]:
%%bash
cd ./lanl/day03
wget -c https://s3-us-gov-west-1.amazonaws.com/unified-host-network-dataset/2017/netflow/netflow_day-03.bz2

In [2]:
headers = ['time', 'duration', 'srcdevice', 'dstdevice', 'protocol',
           'srcport', 'dstport', 'srcpackets', 'dstpackets', 'srcbytes', 'dstbytes']

In [3]:
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs)).isoformat()

In [4]:
dfDay03 = pd.read_csv('./lanl/day03/netflow_day-03.bz2', header=None, names=headers, nrows=500000) # , index_col=0, parse_dates=True, date_parser=dateparse)

In [5]:
len(dfDay03)

500000

In [6]:
dfDay03.tail()

,time,duration,srcdevice,dstdevice,protocol,srcport,dstport,srcpackets,dstpackets,srcbytes,dstbytes
499995,172957,0,Comp208915,Comp275646,17,Port27607,53,1,0,73,0
499996,172957,0,Comp208915,Comp275646,17,Port55333,53,1,0,60,0
499997,172957,0,Comp208915,Comp275646,17,Port74721,53,1,0,65,0
499998,172957,0,Comp208915,Comp275646,17,Port80500,53,1,0,63,0
499999,172957,0,Comp208915,Comp275646,17,Port81522,53,1,0,63,0


In [9]:
dfDay03.to_csv('./lanl/day03/netflow_day-03_partial.csv', header=True, index=False)

In [ ]:
%%bash
cd ./lanl/day03
mkdir -p csv
mkdir -p json
head -1 netflow_day-03_partial.csv > headers.csv
mv netflow_day-03_partial.csv ./csv
mv headers.csv ./csv
cd csv
split -l 10000 netflow_day-03_partial.csv segment_
fnumber=1
files=(*)
for input in $files; do
    printf -v j "%02g" $fnumber;
    echo "working on time_$j.json";
    pwd;
    cat headers.csv $input > time_$j.csv;
    csvjson --stream time_$j.csv > ../json/time_$j.json;
    rm $input time_$j.csv;
    fnumber=$((fnumber + 1));
done;